## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [2]:
# If you didn't download it yet, please get the relevant file now!
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [4]:
import pandas as pd
import numpy as np

file_name="data\data_000637.txt"
with open(file_name) as data:
    n_rows = len(data.readlines())
data=pd.read_csv(file_name, nrows=np.random.randint(1e5,n_rows))
data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
189274,1,1,1,3869202051,1112,11
189275,1,1,2,3869202051,1112,22
189276,1,1,3,3869202051,1112,17
189277,1,1,6,3869202051,1113,25


In [65]:
#the ORBIT_CNT is increased every 'x' BX_COUNTER. x=?

min_diff=min(abs(data['ORBIT_CNT']-data['BX_COUNTER']))
x=data[abs(data['ORBIT_CNT']-data['BX_COUNTER'])==min_diff].iloc[0,4]
print(x)
x_i=data[abs(data['ORBIT_CNT']-data['BX_COUNTER'])==min_diff].index[0]
print(data['ORBIT_CNT'][x_i+1],data['ORBIT_CNT'][x_i+2])


3542
3869200171 3869200172
